In [ ]:
from bs4 import BeautifulSoup
import requests, sys, json
import time

BASE_SAVE_PATH = r'C:\Users\Brandon\git\spotify-recommender-system\data\raw'

In [ ]:
## Request HTML
r = requests.get("http://everynoise.com/engenremap.html")

## Parse Genre HTML Elements
soup = BeautifulSoup(r.content, "html.parser")
allGenreDivs = soup.find_all("div", "genre scanme")

In [ ]:
## Cycle through genres and go to that genres page
artist_mapping = {}
genre_mapping = {}
genreCnt = 0
for genreDiv in allGenreDivs:
    if (genreCnt % 10) == 0:
        print("Pulling genre #" + str(genreCnt))
        sys.stdout.flush()
        
    genre = genreDiv.text.strip().replace('»', '')
    if genre not in genre_mapping:
        genre_mapping[genre] = []
        
    genrePage = "http://everynoise.com/engenremap-" + str.replace(genre," ", "") + ".html"

    ## Pull artists from genre page
    r2 = requests.get(
        genrePage,
        headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'},
        timeout=120
    )
    soup2 = BeautifulSoup(r2.content, "html.parser")
    allArtistDivs = soup2.find_all("div", "genre scanme")

    for artist in allArtistDivs:
        artistName = artist.text.strip().replace('»', '')
        
        if not(artistName.isspace()):
            
            if artistName not in artist_mapping:
                artist_mapping[artistName] = [genre]
            else:
                artist_mapping[artistName].append(genre)
                
            genre_mapping[genre].append(artistName) 
            
    genreCnt = genreCnt+1
    
    time.sleep(2)

In [ ]:
with open(f'{BASE_SAVE_PATH}\genre_artist_mapping.json', 'w') as fp:
    json.dump(genre_mapping, fp)
    
with open(f'{BASE_SAVE_PATH}\artist_genre_mapping.json', 'w') as fp:
    json.dump(artist_mapping, fp)